# About DATA

A manager at the bank is disturbed with more and more customers leaving their credit card services. They would really appreciate if one could predict for them who is gonna get churned so they can proactively go to the customer to provide them better services and turn customers' decisions in the opposite direction

<img src="https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT3VfYG2ZJfvlRVtaora8MkuAnTSr-iiPq52Q&usqp=CAU" width=500 arl="img"><br><br>



There are only 16% customers in the dataset get churned. That is labeled by the column `Attrition_Flag`

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
card_churn = pd.read_csv("/kaggle/input/credit-card-customers/BankChurners.csv")
card_churn.head()

### Drop Uneccessary Columns

In [ ]:
card_churn.drop(["CLIENTNUM","Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1",
                    "Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2"], axis=1, inplace=True)

In [ ]:
card_churn.isnull().sum()

### Change categorical features to numeric

`Attrition Flag`, `Gender`, `Education_Level`, `Marital_Status`, `Income_Category`, `Card_Category`

In [ ]:
# df.Attrition_Flag.unique()
# Output: ['Existing Customer', 'Attrited Customer']

# df.Education_Level.unique()
# Output: ['High School', 'Graduate', 'Uneducated', 'Unknown', 'College',
#        'Post-Graduate', 'Doctorate']

# df.Income_Category.unique()
# Output: ['$60K - $80K', 'Less than $40K', '$80K - $120K', '$40K - $60K',
#        '$120K +', 'Unknown']

# df.Card_Category.unique()
# Output: ['Blue', 'Gold', 'Silver', 'Platinum']

card_churn.Attrition_Flag = card_churn.Attrition_Flag.map({"Attrited Customer": 1,"Existing Customer": 0})
card_churn.Gender = card_churn.Gender.map({"M":1, "F":0})
card_churn.Education_Level = card_churn.Education_Level.map({'High School':2, 'Graduate':4, 'Uneducated':1, 'Unknown':0, 'College':3,
       'Post-Graduate':5, 'Doctorate':6})
card_churn.Marital_Status = card_churn.Marital_Status.map({"Unknown":0, "Single":1, "Married":2, "Divorced":3})
card_churn.Income_Category = card_churn.Income_Category.map({'$60K - $80K':3, 'Less than $40K':1, '$80K - $120K':4, '$40K - $60K':2,
       '$120K +':5, 'Unknown':0})
card_churn.Card_Category = card_churn.Card_Category.map({'Blue':0, 'Gold':1, 'Silver':2, 'Platinum':3})

### Choosing feature to fit into the model

By call `corr()` method, we get a `pd.DataFrame()` object where entries are the correlations of the features. I choosing features which are more related to `Attrition_Flag` to train the model.

In [ ]:
tr_feats = list(card_churn.corr().index[abs(card_churn.corr()["Attrition_Flag"]) >= 0.1])

In [ ]:
tr_feats

## Spliting Data into train, test set

Note that there are only 16% of custumer get churned, so we should carefully in split training data.

In [ ]:
card_churn = card_churn[tr_feats]

#### My splitting IDEA
My Idea is to split the data where `Attrition_Flag == 0` into 6 parts. So I construct `6 batches` having `16% churned` and `13% not`. And giving 6 batches into the model.

So that the features of `not getting churned` will not dominate the `churned`. But there is another problem that those `16% churned` will get trained 6 times more than the other. 

Therefore, I will normalize all the features to `zero mean`, `1 stds`. And then before fitting data into the model, I'll add to each features a random variable `0.1 std`. So it wont change much the features but reduce the bias of `16% churned`.

In [ ]:
card_churn.Attrition_Flag.sum()/card_churn.Attrition_Flag.count()

In [ ]:
# Normalize the features
for feat in tr_feats[1:]:
    card_churn[feat] = (card_churn[feat] - card_churn[feat].mean())/ card_churn[feat].var()

In [ ]:
# tf.random.normal(
#     shape, mean=0.0, stddev=1.0, dtype=tf.dtypes.float32, seed=None, name=None
# )

churned = card_churn[card_churn.Attrition_Flag == 1]
nchurned = card_churn[card_churn.Attrition_Flag==0]

In [ ]:
churned[tr_feats[1:]].agg(lambda x: x+np.random.normal(0,0.1,x.shape))
# churned[tf_feat[1:]]

In [ ]:
l = int(1./6 * nchurned.shape[0])
tr_ratio = 0.9

### Quick evaluate the Idea by KNN model

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

In [ ]:
# My IDEA
def take_patial(churned, nchurned, n=6):
    l = int(nchurned.shape[0]//n)
    trainsets = []
    for i in range(1,n):
        trainsets.append(pd.concat([churned,nchurned[(i-1)*l:i*l]]))
    trainsets.append(pd.concat([churned,nchurned[(n-1)*l:]]))
    return trainsets

def train_test(p, df):
    s = df.shape[0]
    train = df[:int(p*s)]
    val = df[int(p*s):]
    return train, val

def fit(trainset, model, tr_feats=tr_feats):
    x, y = trainset[tr_feats[1:]], trainset[tr_feats[0]]
    noise = np.random.normal(0, 0.1, x.iloc[0].shape[0])
    totrain = x + noise
    model.fit(totrain, y)
    print(model.score(totrain,y))

In [ ]:
tr_churned, val_churned = train_test(tr_ratio, churned)
tr_nchurned, val_nchurned = train_test(tr_ratio, nchurned)

In [ ]:
trainsets = take_patial(tr_churned, tr_nchurned)

In [ ]:
trainsets[0].Attrition_Flag.sum()/trainsets[0].shape[0]

In [ ]:
neighbor = KNeighborsClassifier(n_neighbors=3)

In [ ]:
for trainset in trainsets:
    fit(trainset,neighbor)

In [ ]:
def score(val, model, tr_feats=tr_feats):
    x, y = val[tr_feats[1:]], val[tr_feats[0]]
    print(model.score(x,y))

In [ ]:
score(val_nchurned, neighbor)

In [ ]:
score(pd.concat([val_churned,val_nchurned]),neighbor)

### Normal way

In [ ]:
from sklearn.model_selection import train_test_split

xtrain, xval, ytrain, yval = train_test_split(card_churn[tr_feats[1:]], card_churn.Attrition_Flag)

In [ ]:
neighbor2 = KNeighborsClassifier(n_neighbors=3)

In [ ]:
neighbor2.fit(xtrain, ytrain)

In [ ]:
neighbor2.score(xval, yval)

## Trying with Deep Learning

### Tensorflow

In [ ]:
import tensorflow as tf

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Dense(16, input_shape=(9,)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation="sigmoid")
])

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])

model.summary()

In [ ]:
history = model.fit(xtrain, ytrain,
                    batch_size=128,
                    validation_data=(xval,yval),
                    epochs=10,
                    verbose=2)

In [ ]:
import matplotlib.pyplot as plt
def plot(history, metric):
    plt.plot(history.history[metric])
    plt.plot(history.history['val_'+metric])
    plt.title('model '+metric)
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'validation'], loc='upper left')
    plt.show()

In [ ]:
plot(history, 'loss')

In [ ]:
plot(history, 'acc')